# FastAPI UI Modeling

Let's extend the previous example and provide a more comprehensive picture of how you might structure a larger FastAPI application. This will include additional layers such as services, repositories, middleware, and configurations.

Here's an extended directory layout for a more complex FastAPI project:

In [ ]:
my_fastapi_project/
├── app/
│   ├── __init__.py
│   ├── main.py                         # Entry point for the application
│   ├── api/                            # Contains all API routes and views
│   │   ├── __init__.py
│   │   ├── v1/
│   │       ├── __init__.py
│   │       ├── items.py                # Example module for item-related APIs
│   ├── core/                           # Core settings, configurations, middleware
│   │   ├── __init__.py
│   │   ├── config.py                   # Configuration settings using Pydantic
│   │   ├── middleware.py               # Custom middleware
│   ├── models/                         # Database models using SQLAlchemy or Pydantic
│   │   ├── __init__.py
│   │   ├── item.py                     # Example module for item models
│   ├── db/                             # Database setup and sessions
│   │   ├── __init__.py
│   │   ├── base.py                     # Base model for SQLAlchemy
│   │   ├── session.py                  # Database session creation
│   ├── services/                       # Business logic of your application
│   │   ├── __init__.py
│   │   ├── item_service.py             # Example service for items
│   ├── repositories/                   # Data access layer
│   │   ├── __init__.py
│   │   ├── item_repository.py          # Example repository for item operations
│   ├── schemas/                        # Pydantic models for request and response bodies
│       ├── __init__.py
│       ├── item_schema.py              # Example schema for items
└── requirements.txt                    # Dependencies


**Main Application (`app/main.py`)**  
This remains the entry point where you set up the application and include routers:


In [ ]:
from fastapi import FastAPI
from app.api.v1 import items
from app.core.middleware import custom_middleware

app = FastAPI()

# Include middleware
app.add_middleware(custom_middleware)

# Include the router for versioned API
app.include_router(items.router, prefix="/v1/items")

@app.get("/")
def read_root():
    return {"message": "Welcome to My FastAPI Application"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=True)


**Configuration (`app/core/config.py`)**  
Manage configuration settings using Pydantic:


In [ ]:
from pydantic import BaseSettings

class Settings(BaseSettings):
    app_name: str = "My FastAPI App"
    admin_email: str = "admin@example.com"
    database_url: str

    class Config:
        env_file = ".env"

settings = Settings()


**Middleware (`app/core/middleware.py`)**  
Define custom middleware if needed:


In [ ]:
from starlette.middleware.base import BaseHTTPMiddleware
from starlette.types import ASGIApp, Receive, Scope, Send

class custom_middleware(BaseHTTPMiddleware):
    async def dispatch(self, request: Request, call_next):
        response = await call_next(request)
        response.headers["X-Custom-Header"] = "Value"
        return response


**Database Models (`app/models/item.py`)**  
Define your data models (using SQLAlchemy in this case):


In [ ]:
from sqlalchemy import Column, Integer, String, Float, Boolean
from app.db.base import Base

class Item(Base):
    __tablename__ = "items"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    description = Column(String, index=True)
    price = Column(Float)
    on_offer = Column(Boolean)


**Database Session (`app/db/session.py`)**  
Setup your database session:

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from app.core.config import settings

DATABASE_URL = settings.database_url

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()


**Modify API Router to Inherit from Base Class**  
Now, modify your existing API router to inherit from this base class:


In [ ]:
# app/api/dependencies.py

from sqlalchemy.orm import Session
from app.db.session import SessionLocal

class DBDependency:
    def get_db(self) -> Session:
        db = SessionLocal()
        try:
            yield db
        finally:
            db.close()


In [ ]:
# app/api/v1/items.py

from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
from typing import List

from app.api.dependencies import DBDependency
from app.schemas.item_schema import Item, ItemCreate, ItemUpdate
from app.services.item_service import ItemService

router = APIRouter()

class ItemAPI(DBDependency):

    @router.post("/", response_model=Item)
    def create_item(self, item: ItemCreate, db: Session = Depends(self.get_db)):
        return ItemService.create_item(db, item)

    @router.get("/", response_model=List[Item])
    def get_items(self, db: Session = Depends(self.get_db)):
        return ItemService.get_all_items(db)

    @router.get("/{item_id}", response_model=Item)
    def get_item(self, item_id: int, db: Session = Depends(self.get_db)):
        db_item = ItemService.get_item(db, item_id)
        if db_item is None:
            raise HTTPException(status_code=404, detail="Item not found")
        return db_item

    @router.put("/{item_id}", response_model=Item)
    def update_item(self, item_id: int, item: ItemUpdate, db: Session = Depends(self.get_db)):
        db_item = ItemService.update_item(db, item_id, item)
        if db_item is None:
            raise HTTPException(status_code=404, detail="Item not found")
        return db_item

    @router.delete("/{item_id}")
    def delete_item(self, item_id: int, db: Session = Depends(self.get_db)):
        success = ItemService.delete_item(db, item_id)
        if not success:
            raise HTTPException(status_code=404, detail="Item not found")
        return {"message": "Item deleted"}

# Instantiate the class and set routes
item_api = ItemAPI()
router.add_api_route("/", item_api.create_item, methods=["POST"])
router.add_api_route("/", item_api.get_items, methods=["GET"])
router.add_api_route("/{item_id}", item_api.get_item, methods=["GET"])
router.add_api_route("/{item_id}", item_api.update_item, methods=["PUT"])
router.add_api_route("/{item_id}", item_api.delete_item, methods=["DELETE"])


**Repositories (`app/repositories/item_repository.py`)**  
Create a repository for data access operations:


In [ ]:
from sqlalchemy.orm import Session
from app.models.item import Item
from app.schemas.item_schema import ItemCreate, ItemUpdate

class ItemRepository:

    @staticmethod
    def get_item(db: Session, item_id: int):
        return db.query(Item).filter(Item.id == item_id).first()

    @staticmethod
    def create_item(db: Session, item: ItemCreate):
        db_item = Item(**item.dict())
        db.add(db_item)
        db.commit()
        db.refresh(db_item)
        return db_item

    @staticmethod
    def update_item(db: Session, db_item: Item, item_update: ItemUpdate):
        for key, value in item_update.dict().items():
            setattr(db_item, key, value)
        db.commit()
        db.refresh(db_item)
        return db_item

    @staticmethod
    def delete_item(db: Session, item_id: int):
        db_item = db.query(Item).filter(Item.id == item_id).first()
        if db_item:
            db.delete(db_item)
            db.commit()
            return True
        return False


**Services (`app/services/item_service.py`)**  
Place your business logic here:


In [ ]:
from sqlalchemy.orm import Session
from app.repositories.item_repository import ItemRepository
from app.schemas.item_schema import ItemCreate, ItemUpdate

class ItemService:

    @staticmethod
    def get_item(db: Session, item_id: int):
        return ItemRepository.get_item(db, item_id)

    @staticmethod
    def create_item(db: Session, item: ItemCreate):
        return ItemRepository.create_item(db, item)

    @staticmethod
    def update_item(db: Session, item_id: int, item_update: ItemUpdate):
        db_item = ItemRepository.get_item(db, item_id)
        if not db_item:
            return None
        return ItemRepository.update_item(db, db_item, item_update)

    @staticmethod
    def delete_item(db: Session, item_id: int):
        return ItemRepository.delete_item(db, item_id)


**Schemas (`app/schemas/item_schema.py`)**  
Define your Pydantic schemas:


In [ ]:
from pydantic import BaseModel

class ItemBase(BaseModel):
    name: str
    description: str
    price: float
    on_offer: bool

class ItemCreate(ItemBase):
    pass

class ItemUpdate(ItemBase):
    pass

class Item(ItemBase):
    id: int

    class Config:
        orm_mode = True


**API Module (`app/api/v1/items.py`)**  
Connect everything in your API endpoints:


**Abstract Base Class for Database Operations**  
Define the base class `DBDependency` in the `app/api/dependencies.py` file:


In [ ]:
# app/api/dependencies.py

from sqlalchemy.orm import Session
from app.db.session import SessionLocal

class DBDependency:
    def get_db(self):
        db = SessionLocal()
        try:
            yield db
        finally:
            db.close()


**Modify API Router to Inherit from Base Class**  
Modify your existing API router to inherit from this base class:


In [ ]:
# app/api/v1/items.py

from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
from typing import List

from app.api.dependencies import DBDependency
from app.schemas.item_schema import Item, ItemCreate, ItemUpdate
from app.services.item_service import ItemService

router = APIRouter()

class ItemAPI(DBDependency):

    @router.post("/", response_model=Item)
    def create_item(self, item: ItemCreate, db: Session = Depends(self.get_db)):
        return ItemService.create_item(db, item)

    @router.get("/", response_model=List[Item])
    def get_items(self, db: Session = Depends(self.get_db)):
        return ItemService.get_all_items(db)

    @router.get("/{item_id}", response_model=Item)
    def get_item(self, item_id: int, db: Session = Depends(self.get_db)):
        db_item = ItemService.get_item(db, item_id)
        if db_item is None:
            raise HTTPException(status_code=404, detail="Item not found")
        return db_item

    @router.put("/{item_id}", response_model=Item)
    def update_item(self, item_id: int, item: ItemUpdate, db: Session = Depends(self.get_db)):
        db_item = ItemService.update_item(db, item_id, item)
        if db_item is None:
            raise HTTPException(status_code=404, detail="Item not found")
        return db_item

    @router.delete("/{item_id}")
    def delete_item(self, item_id: int, db: Session = Depends(self.get_db)):
        success = ItemService.delete_item(db, item_id)
        if not success:
            raise HTTPException(status_code=404, detail="Item not found")
        return {"message": "Item deleted"}

# Instantiate the class and set routes
item_api = ItemAPI()
router.add_api_route("/", item_api.create_item, methods=["POST"])
router.add_api_route("/", item_api.get_items, methods=["GET"])
router.add_api_route("/{item_id}", item_api.get_item, methods=["GET"])
router.add_api_route("/{item_id}", item_api.update_item, methods=["PUT"])
router.add_api_route("/{item_id}", item_api.delete_item, methods=["DELETE"])


**Explanation**

- **DBDependency Base Class:** The `DBDependency` class provides the `get_db` method to manage the database session lifecycle.

- **ItemAPI Class:** This class inherits from `DBDependency` and defines the endpoint handlers using methods. Each method uses the `get_db` dependency provided by the base class.

- **Router Configuration:** We instantiate the `ItemAPI` class and add the routes to the router.

By organizing your code this way, you reduce repetition and improve maintainability, as common logic such as database session handling is centralized in one place.


**Installation and Running**  
Add SQLAlchemy to your `requirements.txt`:


In [ ]:
fastapi
uvicorn
sqlalchemy
pydantic


Install the dependencies:

In [ ]:
pip install -r requirements.txt


Run your application:

In [ ]:
uvicorn app.main:app --reload

## Conclusion  
This enhanced structure provides a clear separation of concerns by dividing the project into routers, services, repositories, schemas, and configurations. It allows you to manage and scale your codebase effectively while adhering to best practices.
